In [ ]:
# SQL script for database creation.

schema.sql
CREATE TABLE patients (
    patient_id SERIAL PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    date_of_birth DATE,
    gender VARCHAR(10),
    phone_number VARCHAR(15),
    address TEXT,
    neurology_history TEXT
);

CREATE TABLE doctors (
    doctor_id SERIAL PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    specialization VARCHAR(100) DEFAULT 'Neurology',
    phone_number VARCHAR(15),
    license_number VARCHAR(50)
);

CREATE TABLE neurological_assessments (
    assessment_id SERIAL PRIMARY KEY,
    patient_id INT REFERENCES patients(patient_id),
    doctor_id INT REFERENCES doctors(doctor_id),
    assessment_date DATE,
    findings TEXT,
    diagnosis TEXT
);

CREATE TABLE appointments (
    appointment_id SERIAL PRIMARY KEY,
    patient_id INT REFERENCES patients(patient_id),
    doctor_id INT REFERENCES doctors(doctor_id),
    appointment_date DATE,
    status VARCHAR(20)
);

CREATE TABLE prescriptions (
    prescription_id SERIAL PRIMARY KEY,
    patient_id INT REFERENCES patients(patient_id),
    doctor_id INT REFERENCES doctors(doctor_id),
    medication TEXT,
    dosage VARCHAR(100),
    prescription_date DATE
);

CREATE TABLE billing (
    bill_id SERIAL PRIMARY KEY,
    patient_id INT REFERENCES patients(patient_id),
    amount DECIMAL(10, 2),
    payment_status VARCHAR(20),
    payment_date DATE
);


In [ ]:
# ETL Pipeline for Neurology Clinic Data

#ETL (Extract, Transform, Load) pipeline for handling patient and appointment data from the Clinic at Temple University. I'll extract data from CSV files, transform it to ensure consistent formats, and load it into a PostgreSQL database.


### 1. **Environment Setup**
#First, we'll import the necessary libraries and establish a connection to the PostgreSQL database.


In [ ]:
import pandas as pd
import psycopg2

def extract_data(file_path):
    return pd.read_csv(file_path)

def transform_data(df):
    df['appointment_date'] = pd.to_datetime(df['appointment_date'], format='%Y-%m-%d')
    return df

def load_data(df, table_name, conn):
    cursor = conn.cursor()
    for _, row in df.iterrows():
        cursor.execute(f"INSERT INTO {table_name} VALUES ({','.join(['%s'] * len(row))})", tuple(row))
    conn.commit()

def run_etl():
    conn = psycopg2.connect(
        dbname="neurology_clinic_db", user="username", password="password", host="localhost", port="5432"
    )
    patients_df = extract_data('patients.csv')
    appointments_df = extract_data('appointments.csv')
    
    transformed_patients = transform_data(patients_df)
    transformed_appointments = transform_data(appointments_df)
    
    load_data(transformed_patients, 'patients', conn)
    load_data(transformed_appointments, 'appointments', conn)

if __name__ == '__main__':
    run_etl()


In [ ]:
# Breakdown of these steps

In [ ]:
# Import necessary libraries
import pandas as pd
import psycopg2

# Establish a connection to PostgreSQL (replace with user credentials)
conn = psycopg2.connect(
    dbname="neurology_clinic_db", 
    user="username", 
    password="password", 
    host="localhost", 
    port="5432"
)

cursor = conn.cursor()

In [ ]:
### 2. **Extract Phase**
now I extract the data from CSV files into Pandas DataFrames.

In [ ]:
# Extract data from CSV files
patients_df = pd.read_csv('patients.csv')
appointments_df = pd.read_csv('appointments.csv')

# Display extracted data
print("Patients Data:")
display(patients_df.head())

print("\nAppointments Data:")
display(appointments_df.head())

In [ ]:
### 3. **Transform Phase**
I'll perform transformations on the extracted data, such as ensuring date formats are consistent and filling in missing values.

In [ ]:
# Convert appointment_date to datetime
appointments_df['appointment_date'] = pd.to_datetime(appointments_df['appointment_date'], format='%Y-%m-%d')

# Handle missing values (if any)
patients_df.fillna("Unknown", inplace=True)
appointments_df.fillna("Unknown", inplace=True)

# Display transformed data
print("\nTransformed Patients Data:")
display(patients_df.head())

print("\nTransformed Appointments Data:")
display(appointments_df.head())

In [ ]:
### 4. **Load Phase**
load the transformed data into the PostgreSQL database.

In [ ]:
# Load patients data into the database
for _, row in patients_df.iterrows():
    cursor.execute("""
        INSERT INTO patients (first_name, last_name, date_of_birth, gender, phone_number, address, neurology_history)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))

# Load appointments data into the database
for _, row in appointments_df.iterrows():
    cursor.execute("""
        INSERT INTO appointments (patient_id, doctor_id, appointment_date, status)
        VALUES (%s, %s, %s, %s)
    """, tuple(row))

# Commit changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Data loaded into the database successfully.")

In [ ]:
## Performance tunning

In [ ]:
-- indexing.sql
CREATE INDEX idx_patient_last_name ON patients(last_name);
CREATE INDEX idx_appointment_date ON appointments(appointment_date);

-- Example query before optimization
SELECT * FROM patients WHERE last_name = 'Smith';

-- Optimized query after indexing
EXPLAIN ANALYZE SELECT * FROM patients WHERE last_name = 'Smith';

In [ ]:
## NoSQL build using MongoDB

In [ ]:
// mongodb_script.js
db.createCollection("patients");

db.patients.insertMany([
    {
        first_name: "John",
        last_name: "Doe",
        age: 45,
        conditions: ["diabetes", "hypertension"],
        appointments: [
            { date: "2024-10-01", doctor: "Dr. Smith", status: "completed" }
        ]
    },
    {
        first_name: "Jane",
        last_name: "Doe",
        age: 50,
        conditions: ["asthma"],
        appointments: [
            { date: "2024-09-21", doctor: "Dr. Jones", status: "pending" }
        ]
    }
]);

In [ ]:
#Deploy database on AWS using RDS

In [ ]:
# main.tf
provider "aws" {
  region = "us-east-1"
}

resource "aws_db_instance" "healthcare_db" {
  allocated_storage    = 20
  engine               = "postgres"
  engine_version       = "12.3"
  instance_class       = "db.t2.micro"
  name                 = "healthcare_db"
  username             = "admin"
  password             = "password123"
  parameter_group_name = "default.postgres12"
  publicly_accessible  = true
  skip_final_snapshot  = true
}

In [ ]:
#CI/CD pipeline setup for automating database
#dockerfile for setting up database in docker container, jenkinsfile for automating deployments

In [ ]:
// Jenkinsfile
pipeline {
    agent any
    stages {
        stage('Build') {
            steps {
                echo 'Building Docker image'
                sh 'docker build -t healthcare-db .'
            }
        }
        stage('Deploy') {
            steps {
                echo 'Deploying database schema'
                sh 'docker run healthcare-db psql -U postgres -f /scripts/schema.sql'
            }
        }
    }
}